TCC MBA USP EsalQ - Data Science Analytics – 2023 - Maicon Melara

Importando Bibliotecas

In [ ]:
# pip install -U scikit-learnimport json
import pandas as pd # lib pandas
import numpy as np # lib numpy
import datetime # lib datetime

import matplotlib.pyplot as plt # lib para utilização dos gráficos
import seaborn as sns # lib para exibir dados estátisticos

# import numpy as np from scipy import stats
# stats.mode(np.ones(3))[0]


# Necessário para visualização automática dos gráficos no Jupyter
%matplotlib inline

from pathlib import Path
from pandas import json_normalize # package for flattening json in pandas df
from scipy import stats

# lib sklearn
from sklearn.model_selection import train_test_split # lib para definir os dados de treino e de teste
from sklearn.neighbors import KNeighborsClassifier # lib classe KNeighborsClassifier - Classificação 
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score,roc_curve
from sklearn import preprocessing # Import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier # lib classe RandomForestClassifier - Classificação 
# from yellowbrick.classifier import ClassificationReport

In [ ]:
import json

Carregando Base de Dados "data-bikes.json"

In [ ]:
p = Path(r'C:\Users\maicon.melara\Documents\Python\data\Bases\data-bikes.json')
with p.open('r', encoding='utf-8') as f:
    data = json.loads(f.read())

df_bikes = json_normalize(data) 

df_bikes.info()

Carregando Base de Dados "data-bikes-fipe.json"

In [ ]:
p = Path(r'C:\Users\maicon.melara\Documents\Python\data\Bases\data-bikes-fipe.json')
with p.open('r', encoding='utf-8') as f:
    data = json.loads(f.read())
                      
df_bikes_fipe = json_normalize(data)  

df_bikes_fipe.info()

Unindo as bases de dados "data-bikes" e "data-bikes-fipe"

In [ ]:
df_bikes = df_bikes.join(df_bikes_fipe.set_index('UniqueId')[['Fipe']], on='UniqueId')

Edição de alguns campos da Base - Deletando

In [ ]:
del df_bikes['Seller.Id'] 
del df_bikes['Seller.SellerType'] 
del df_bikes['Seller.AdType.id']
del df_bikes['Seller.AdType.Value'] 
del df_bikes['Seller.CarDelivery'] 
del df_bikes['Seller.TrocaComTroco']
del df_bikes['Seller.ExceededPlan'] 
del df_bikes['Seller.BudgetInvestimento'] 
del df_bikes['Seller.DealerScore'] 
del df_bikes['Seller.City'] 
del df_bikes['Seller.State'] 
del df_bikes['Seller.FantasyName']  

In [ ]:
del df_bikes['Specification.Model.id'] 
del df_bikes['Specification.Make.id'] 
del df_bikes['Specification.Color.IdPrimary']
del df_bikes['Specification.VehicleAttributes'] 
del df_bikes['Specification.YearFabrication']
del df_bikes['Specification.BodyType'] 
del df_bikes['Specification.Title']   
del df_bikes['Specification.CubicCentimeter']     
del df_bikes['Specification.Engine.Id']           
del df_bikes['Specification.Engine.Value']       
del df_bikes['Specification.Break.Id']            
del df_bikes['Specification.Break.Value']         
del df_bikes['Specification.Charge.Id']          
del df_bikes['Specification.Charge.Value']     
del df_bikes['Specification.Cooling.Id']        
del df_bikes['Specification.Cooling.Value']      
del df_bikes['Specification.Shift.Id']         
del df_bikes['Specification.Shift.Value']      
del df_bikes['Specification.Starter.Id']         
del df_bikes['Specification.Starter.Value']            
del df_bikes['Specification.Color.Secundary']   
del df_bikes['Specification.Color.IdSecundary']
del df_bikes['Prices.SearchPrice']

Renomeando alguns campos - Rename

In [ ]:
df_bikes.rename({'Specification.Make.Value': 'Make',
                'Specification.Model.Value': 'Model',
              #  'Specification.Version.Value': 'Version',
                'Specification.YearModel': 'YearModel',
                'Specification.Odometer': 'Odometer',
              #  'Specification.Armored': 'Armored',
                'Prices.Price': 'Price',
                'Specification.Color.Primary': 'Color',
                               }, axis=1, inplace=True)

In [ ]:
df_bikes.rename({'Seller.OnlyOwner': 'OnlyOwner',}, axis=1, inplace=True) 

Convertendo alguns DTypes - Tipos de Dados

In [45]:
df_bikes['YearModel'] = df_bikes['YearModel'].astype('int64')
df_bikes['Make'] = df_bikes['Make'].astype('string')
df_bikes['Model'] = df_bikes['Model'].astype('string') 

Verifica se existe algum valor nulo após o tratamento dos dados

In [55]:
df_bikes.isnull().sum()

UniqueId     0
Licensed     0
IPVApaid     0
Warranty     0
Make         0
Model        0
YearModel    0
Odometer     0
Color        0
OnlyOwner    0
Price        0
GoodDeal     0
Fipe         0
dtype: int64

Exibe a dimensão do DataFrame - linhas x colunas

In [57]:
df_bikes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504 entries, 0 to 503
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   UniqueId   504 non-null    int64  
 1   Licensed   504 non-null    bool   
 2   IPVApaid   504 non-null    bool   
 3   Warranty   504 non-null    bool   
 4   Make       504 non-null    string 
 5   Model      504 non-null    string 
 6   YearModel  504 non-null    int64  
 7   Odometer   504 non-null    float64
 8   Color      504 non-null    object 
 9   OnlyOwner  504 non-null    object 
 10  Price      504 non-null    float64
 11  GoodDeal   504 non-null    object 
 12  Fipe       504 non-null    object 
dtypes: bool(3), float64(2), int64(2), object(4), string(2)
memory usage: 41.0+ KB


Exibe a dimensão do DataFrame - Qtd de linhas x colunas

In [59]:
df_bikes.shape

(504, 13)

Exibe o valor total de motos que estão acima ao da tabela Fipe e IPVA Pago

In [72]:
bikes = (df_bikes['Price'] > df_bikes['Fipe'])

In [73]:
df_bikes[bikes].shape[0]

41

Exibe o valor total de motos que estão abaixo ou igual ao da tabela Fipe e IPVA Pago

In [74]:
bikes = (df_bikes['Price'] <= df_bikes['Fipe'])

In [75]:
df_bikes[bikes].shape[0]

463